In [41]:
import re
from collections import Counter

In [42]:
import warnings
warnings.filterwarnings("ignore")

In [43]:
def words(text):
    return re.findall(r'\w+', text.lower())

In [44]:
words ('EBook of The Adventures of Sherlock Holmes')

['ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes']

In [45]:
dictionary = Counter(words(open('data/big.txt').read()))

In [46]:
dictionary

Counter({'the': 79809,
         'project': 288,
         'gutenberg': 263,
         'ebook': 87,
         'of': 40024,
         'adventures': 17,
         'sherlock': 101,
         'holmes': 467,
         'by': 6735,
         'sir': 177,
         'arthur': 34,
         'conan': 4,
         'doyle': 5,
         '15': 47,
         'in': 22023,
         'our': 1065,
         'series': 128,
         'copyright': 51,
         'laws': 233,
         'are': 3630,
         'changing': 43,
         'all': 4143,
         'over': 1282,
         'world': 362,
         'be': 6155,
         'sure': 123,
         'to': 28765,
         'check': 38,
         'for': 6941,
         'your': 1279,
         'country': 423,
         'before': 1362,
         'downloading': 5,
         'or': 5352,
         'redistributing': 7,
         'this': 4063,
         'any': 1203,
         'other': 1501,
         'header': 7,
         'should': 1297,
         'first': 1174,
         'thing': 303,
         'seen': 444,
  

Find the correction c, out of all possible candidate corrections, that maximizes the probability that c is the intended correction, given the original word w

Bayes' Theorem

**P(c|w) = P(c) P(w|c) / P(w)**

- **c** - Correction


- **w** - Word


- **P(c)** - The probability that c appears as a word of English text. <br>For example, occurrences of "the" make up about 7% of English text, so we should have P(the) = 0.07


- **P(w|c)** - The probability that w would be typed in a text when the author meant c. <br>For example, P(teh|the) is relatively high, but P(theeexyz|the) would be very low

In [47]:
def P(word, N=sum(dictionary.values())):
    "Probability of `word`"
    return dictionary[word]/N

In [48]:
def correction(word):
    "Most probable spelling correction for word"
    return max(candidates(word), key=P)

All known words of edit distance 1 are infinitely more probable than known words of edit distance 2, and infinitely less probable than a known word of edit distance 0

In [49]:
def candidates(word):
    "Generate possible spelling correction for words"
    return(known([word]) or known(edits1(word)) or \
          known(edits2(word)) or [word])

we don't need to multiply by a P(w|c) factor, because every candidate at the chosen priority will have the same probability

In [50]:
def known(words):
    "The subset of words that appear in the dictionary"
    return set(w for w in words if w in dictionary)

**Sime Edit** - deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter)

In [51]:
def edits1(word):
    "All edits that are one edit away from word"
    letters ='abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word)+1)]
    deletes = [L + R[1:] for L,R in splits if R]
    transposes = [L + R[1] +R[0]+R[2:] for L, R in splits if len(R)>1]
    replaces = [L +c +R[1:] for L,R in splits if R for c in letters]
    inserts = [L+ c+R  for L,R in splits for c in letters]
    return set(deletes + transposes +replaces +inserts)

In [52]:
len(edits1('inention'))

442

In [53]:
known(edits1('inention'))

{'intention', 'invention'}

In [54]:
def edits2(word):
    "two edits away from word"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [55]:
len(set(edits2('inention')))

90847

In [56]:
candidates('acient')

{'accent', 'ancient'}

In [57]:
known(edits2('inention'))

{'ention',
 'inaction',
 'inanition',
 'inception',
 'infection',
 'ingestion',
 'injection',
 'insertion',
 'intention',
 'intentions',
 'inunction',
 'invention',
 'inventions',
 'mention'}

In [58]:
len(dictionary)

32198

In [59]:
sum(dictionary.values())

1115585

In [60]:
P('how')

0.001178753748033543

In [61]:
P('invention')

8.067516146237176e-06

In [62]:
dictionary.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [63]:
correction('speling') # Missing letter

'spelling'

In [64]:
correction('korrectud') #Two letter correction

'corrected'

In [65]:
correction('inetion') #Missing two letters

'infection'

In [66]:
['1','2','3'] or ['4','5']

['1', '2', '3']

### Test

In [67]:
def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
    assert Counter(words('This is a test. 123; A TEST this is.')) == (
           Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
    assert len(dictionary) == 32198
    assert sum(dictionary.values()) == 1115585
#     assert dictionary.most_common(10) == [
#      ('the', 79808),
#      ('of', 40024),
#      ('and', 38311),
#      ('to', 28765),
#      ('in', 22020),
#      ('a', 21124),
#      ('that', 12512),
#      ('he', 12401),
#      ('was', 11410),
#      ('it', 10681)]
#    assert dictionary['the'] == 79808
    assert P('quintessential') == 0
    assert 0.07 < P('the') < 0.08
    return 'unit_tests pass'

In [68]:
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in dictionary)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, dictionary[w], right, dictionary[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))

In [69]:
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [70]:
print(unit_tests())
spelltest(Testset(open('data/spell-testset1.txt'))) # Development set
spelltest(Testset(open('data/spell-testset2.txt'))) # Final test set

unit_tests pass
75% of 270 correct (6% unknown) at 40 words per second 
68% of 400 correct (11% unknown) at 36 words per second 
